In [1]:
include("/home/krzysztof/Dokumenty/badania_iitis/tensors_symetric/tensor calculations/calc&codes/sym_tensor/src/Boxtensors.jl")  
include("Test.jl")
using Test

In [2]:
include("/home/krzysztof/Dokumenty/badania_iitis/tensors_symetric/tensor calculations/calc&codes/sym_tensor/src/Boxtensors.jl") 

Boxtensors

In [2]:
using Boxtensors
importall Boxtensors

In [15]:
data = clcopulagen(20000, 16);
#using PyPlot
#plot3D(data[:,1], data[:,10], data[:,15], "o")
;

In [4]:
@time c = snaivecumulant(data, 6)

1487.891777 seconds (6.15 G allocations: 276.517 GB, 8.84% gc time)


Dict{ASCIIString,Any} with 8 entries:
  "c2" => 4x4 Array{Float64,2}:…
  "c8" => 4x4x4x4x4x4x4x4 Array{Float64,8}:…
  "c9" => 4x4x4x4x4x4x4x4x4 Array{Float64,9}:…
  "c3" => 4x4x4 Array{Float64,3}:…
  "c7" => 4x4x4x4x4x4x4 Array{Float64,7}:…
  "c4" => 4x4x4x4 Array{Float64,4}:…
  "c6" => 4x4x4x4x4x4 Array{Float64,6}:…
  "c5" => 4x4x4x4x4 Array{Float64,5}:…

In [17]:
@time c2, c3, c4, c5, c6 = cumulants(6, data, 4);

327.101555 seconds (946.42 M allocations: 693.939 GB, 16.93% gc time)


In [74]:
@time  cf = get_diff(data, 10);

347.649006 seconds (3.71 M allocations: 1.734 GB, 0.08% gc time)


In [13]:
#138.701652 seconds
#1627.469713 seconds (6.28 G allocations: 595.513 GB, 4.33% gc time)
# 543.527439 seconds (1.89 G allocations: 870.856 GB, 10.93% gc time)
#test = convert(Array, c10) - cf[10]
test = c["c9"] - convert(Array, c9)
maximum(abs(test))

5.434230843093246e-11

In [ ]:
@time bs+bs1;
@time bs+2;
@time st + st1;
@time st + 2;
@time convert(Array, bs);
@time vec(bs);
@time vec(st);
@time bs.*bs1;
@time bs+bs1;
@time bs./bs1;
@time st.*st1;
@time st+st1;
@time st./st1;
@time modemult(bs, c1', 4)
@time Tensors.modemult(st,c1',4);
@time bcssclass(bs, c1')